In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision import models

from sklearn.model_selection import train_test_split


#  Data prep

In [2]:
data_dir = "/kaggle/input/dog-and-cat-classification-dataset/PetImages"

# custom_dataset = datasets.ImageFolder(root=data_dir, transforms = transform)
full_dataset = datasets.ImageFolder(root=data_dir)

indices = list(range(len(full_dataset)))

In [3]:
len(indices)


24998

In [4]:
train_idx, test_idx = train_test_split(indices, test_size = 0.2, random_state=41, stratify = full_dataset.targets)

In [5]:
train_dataset = Subset(full_dataset,train_idx)
test_dataset = Subset(full_dataset,test_idx)

# Applying Transforms

In [6]:
train_dataset.dataset.transform = transforms.Compose([
    transforms.Resize((244,244)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

test_dataset.dataset.transform = transforms.Compose([
    transforms.Resize((244,244)),
    transforms.ToTensor()
])

In [7]:
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
test_loader = DataLoader(train_dataset, batch_size = 32, shuffle = False)

# Lets Create our Resnet Model function

In [8]:
def resnetModel(num_classes):

    model = models.resnet18(weights = models.ResNet18_Weights.DEFAULT)

    for params in model.parameters():
        params.requires_grad = False


    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs,num_classes)

    return model

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
model = resnetModel(2).to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 224MB/s]


In [10]:
device

device(type='cuda', index=0)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr = 0.001)

# Lets Train the model

In [ ]:
epochs = 10
losses = []

print(f"Training Started ON {device}")

for epoch in range(epochs):
    
    model.train()
    running_loss = 0.0

    for X, y in train_loader:

        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()

        output = model(X)
        
        loss = criterion(output,y)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()

        avg_loss = running_loss/len(train_loader)
        losses.append(avg_loss)

    print(f"Epochs : {epoch} / {epochs} , Loss : {running_loss/len(train_loader):.4f}")

    print(f"Precision Loss : {loss.item():.5f}")

print("Training Completed")
        
        

Training Started ON cuda:0
Epochs : 0 / 10 , Loss : 0.1152
Precision Loss : 0.08572
Epochs : 1 / 10 , Loss : 0.0772
Precision Loss : 0.09599
Epochs : 2 / 10 , Loss : 0.0677
Precision Loss : 0.01846


In [ ]:
torch.save(model.state_dict(), 'CatsnDogs.pt')

# Lets Evaluate

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
model.eval()

y_pred = []
y_true = []

print('Evalution Started')

with torch.no_grad():

    for X,y in test_loader:

        X,y = X.to(device), y.to(device)

        outputs = model(X)

        _,predictions = torch.max(outputs,1)

        # Save results (Move back to CPU to use with Sklearn)
        y_true.extend(y.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())


In [ ]:
print(f"Accuracy score : {accuracy_score(y_true,y_pred):.4f}")
print("Classification Report: \n")
print(classification_report(y_true,y_pred, target_names=['Cat', 'Dog']))

cm = confusion_matrix(y_true,y_pred)


In [ ]:
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt = 'd', cmap="Blues",
           xticklabels = ['Cat','Dog'],
           yticklabels = ['Cat', 'Dog'])
plt.xlabel('Predicted')
plt.ylabel('True')


# Lets Test the Model

In [ ]:
import torch
import torch.nn as nn
from PIL import Image
from torchvision import models, transforms

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

def load_trained_model(model_path,num_classes):

    model = models.resnet18()
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)

    model.load_state_dict(torch.load(model_path, map_location= device))
    model.to(device)
    model.eval()

    return model

transform = transforms.Compose([
    transforms.Resize((244,244)),
    transforms.ToTensor(),
    # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def predict_image(img_path, model):

    img = Image.open(img_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0)

    with torch.no_grad():

        output = model(img_tensor)
        probabilities = torch.softmax(output, dim=1)  # Get probabilities
        _,pred = torch.max(output,1)

         # Print probabilities for debugging
        print(f"Probabilities: Cat={probabilities[0][0]:.4f}, Dog={probabilities[0][1]:.4f}")
        
        
        return "Dog" if pred.item() == 1 else "Cat"

model = load_trained_model(r'X:\Python\ML\Projects\Cats_and_Dogs\CatsnDogs.pt',num_classes=2)
result = predict_image(r'X:\Python\ML\Projects\Cats_and_Dogs\03.jpg',model)
print(f'Prediction : {result}')